In [36]:
# Importing Libraries

# We will be using selenium to automate to navigate to the said page. 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests #Helps in obtaining the requests from the 
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import time
from lxml import html
import requests,json, re

In [37]:
#Set the Main URL
SLEEP_TIME=5
FETCH_DELAY=3
GRAB_FOOD_BASE_URL = 'https://food.grab.com/ph/en/'
QUERY_LOCATION="Chinatown Complex - 335 Smith St, Singapore, 050335"

In [38]:
# Init of Selenium - Chrome WebDriver

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
driver =webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
#Depreciation warning can be ignored, as we are trying to primarily complete the task within the deadline :)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\asus\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
<ipython-input-38-91398b79821b>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver =webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)


In [39]:
# name_elements = driver.find_elements_by_xpath("//div[@class ='ant-layout']" and "//div[@class = 'ant-col-12 ant-col-md-8 ant-col-lg-6']" 
#                                               and "//div[@class ='category___2FjuL']" and "//div[@class ='placeholder___1xbBh categoryImg___3xQyl']"")
                                              
driver.get(GRAB_FOOD_BASE_URL)
                    


In [40]:
url_elements = driver.find_elements_by_xpath("//a[contains(@href, '/ph/en/cuisines/')]")
url_elements

<ipython-input-40-308dba701f97>:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  url_elements = driver.find_elements_by_xpath("//a[contains(@href, '/ph/en/cuisines/')]")


[<selenium.webdriver.remote.webelement.WebElement (session="bbccd685da13e7a6eaf305ac2a46a401", element="8b9f6c1c-b1fe-4fa3-941c-907797284c58")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bbccd685da13e7a6eaf305ac2a46a401", element="27478fc4-219d-4d48-b81a-bca0e4554faa")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bbccd685da13e7a6eaf305ac2a46a401", element="0958cd56-a9b1-40ed-b4de-82c9c6b06867")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bbccd685da13e7a6eaf305ac2a46a401", element="0222c342-9488-448e-ac39-da379fe9f83a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bbccd685da13e7a6eaf305ac2a46a401", element="f4273449-214f-4fba-a5bb-a24a96d85d23")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bbccd685da13e7a6eaf305ac2a46a401", element="31cb6f7e-9393-48bf-8e5f-86ea04e7a0e9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bbccd685da13e7a6eaf305ac2a46a401", element="bc0ec0cb-564f-4b17-b06c-34

In [41]:
main_url_elements_list = []
for url_element in url_elements:
    main_url_elements_list.append(url_element.get_attribute("href"))
main_url_elements_list

['https://food.grab.com/ph/en/cuisines/pizza-delivery/135',
 'https://food.grab.com/ph/en/cuisines/chinese-delivery/45',
 'https://food.grab.com/ph/en/cuisines/beef-delivery/23',
 'https://food.grab.com/ph/en/cuisines/bakery-delivery/6',
 'https://food.grab.com/ph/en/cuisines/american-delivery/3',
 'https://food.grab.com/ph/en/cuisines/japanese-delivery/89',
 'https://food.grab.com/ph/en/cuisines/chicken-delivery/43',
 'https://food.grab.com/ph/en/cuisines/italian-delivery/88',
 'https://food.grab.com/ph/en/cuisines/burgers-delivery/33',
 'https://food.grab.com/ph/en/cuisines/beverages-delivery/24',
 'https://food.grab.com/ph/en/cuisines/kiosk-delivery/95',
 'https://food.grab.com/ph/en/cuisines/milk-tea-delivery/113',
 'https://food.grab.com/ph/en/cuisines/breakfast-brunch-delivery/28',
 'https://food.grab.com/ph/en/cuisines/casual-dining-delivery/39',
 'https://food.grab.com/ph/en/cuisines/asian-delivery/4',
 'https://food.grab.com/ph/en/cuisines/fast-food-delivery/62',
 'https://foo

In [51]:
def urldatagrabber(GRAB_FOOD_BASE_URL):

    driver.get(GRAB_FOOD_BASE_URL)

    i=0
    while True:
        try:
            #Modify this based on internet speed
            time.sleep(SLEEP_TIME)
            button = driver.find_element_by_class_name("ant-btn")
            button.click()

        except Exception as e:
            print("Number of pages scraped: ",i)
            print(e)
            break
        i+=1

        # To test the case scenario, let's assume the page ends after 20 loads. In real life cases, we can let this run for 
        # an extended time period
        if i==5:
            break
    name_elements = driver.find_elements_by_xpath("//div[@class ='ant-layout']" and "//div[@class = 'ant-row-flex RestaurantListRow___1SbZY']" and "//div[@class ='ant-col-24 RestaurantListCol___1FZ8V  ant-col-md-12 ant-col-lg-6']" and "//h6[@class = 'name___2epcT']")
    url_elements = driver.find_elements_by_xpath("//a[contains(@href, '/ph/en/restaurant')]")
    #The first 11 or so restaurants captures seem to be suggestions - 
    #Assuming that they are going to make our dataset polluted, they are being removed

    name_elements = name_elements[10:]

    name_list = []
    for name_element in name_elements:
        name_list.append(name_element.text)

    url_elements_list = []
    for url_element in url_elements:
        url_elements_list.append(url_element.get_attribute("href"))
    url_elements_list = url_elements_list[10:]
    grab_restaurant_details_db = pd.DataFrame(list(zip(name_list,url_elements_list)), columns = ['Restaurant','URL'])
    grab_restaurant_details_db['Latitude'] = ''
    grab_restaurant_details_db['Longitude'] = ''
    grab_restaurant_details_db['Latitude & Longitude'] = ''
    grab_restaurant_details_db
    
    i = 0
    
    for index, row in grab_restaurant_details_db.iterrows():

        grab_restaurant_details_db_url = row[1]
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

        grab_restaurant_details_db_page = requests.get(grab_restaurant_details_db_url, headers=headers)
        soup = BeautifulSoup(grab_restaurant_details_db_page.content, 'html5lib')
        script_text = soup.find('script',id="__NEXT_DATA__").get_text()
        relevant = script_text #removes = and the part before it
    #     print(relevant)
        data = json.loads(relevant) #a dictionary!



        # Method 1


        restaurant_latitude = data['props']['pageProps']['routeState']['location']['latitude']
        restaurant_longitude = data['props']['pageProps']['routeState']['location']['longitude']
        print(row)
        print("OP", restaurant_latitude, restaurant_longitude)
        grab_restaurant_details_db.at[i,'Latitude'] = restaurant_latitude
        grab_restaurant_details_db.at[i,'Longitude'] = restaurant_longitude


        #Method 2

        restaurant_latitude_longitude = data['props']['pageProps']['payload']['latlng']
    #     print(row)
        print(restaurant_latitude_longitude)
        grab_restaurant_details_db.at[i,'Latitude & Longitude'] = restaurant_latitude_longitude


        # I noticed that the latitudes and longitudes were sent from 2 different places, I'm capturing them both

        # introducing a time delay so that the web-site doesn't get concerned by hundreds of rapid requests from one IP address.
        time.sleep(FETCH_DELAY)

        i+=1
    return grab_restaurant_details_db

In [52]:
maindb=pd.DataFrame()
for mainlinks in main_url_elements_list:
    op=urldatagrabber(mainlinks)
    print(op)
    maindb = maindb.append(op, ignore_index=True)
    
# The progrm shows that it has been terminated only bacause I explicitly stopped it. 
# But if the code is run with lesser sleeptime, all the restaurants can be successfully scraped using this

<ipython-input-51-833086e0a04f>:10: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  button = driver.find_element_by_class_name("ant-btn")


Number of pages scraped:  2
Message: no such element: Unable to locate element: {"method":"css selector","selector":".ant-btn"}
  (Session info: chrome=102.0.5005.115)
Stacktrace:
Backtrace:
	Ordinal0 [0x00DDD953+2414931]
	Ordinal0 [0x00D6F5E1+1963489]
	Ordinal0 [0x00C5C6B8+837304]
	Ordinal0 [0x00C89500+1021184]
	Ordinal0 [0x00C8979B+1021851]
	Ordinal0 [0x00CB6502+1205506]
	Ordinal0 [0x00CA44E4+1131748]
	Ordinal0 [0x00CB4812+1198098]
	Ordinal0 [0x00CA42B6+1131190]
	Ordinal0 [0x00C7E860+976992]
	Ordinal0 [0x00C7F756+980822]
	GetHandleVerifier [0x0104CC62+2510274]
	GetHandleVerifier [0x0103F760+2455744]
	GetHandleVerifier [0x00E6EABA+551962]
	GetHandleVerifier [0x00E6D916+547446]
	Ordinal0 [0x00D75F3B+1990459]
	Ordinal0 [0x00D7A898+2009240]
	Ordinal0 [0x00D7A985+2009477]
	Ordinal0 [0x00D83AD1+2046673]
	BaseThreadInitThunk [0x76E86739+25]
	RtlGetFullPathName_UEx [0x777F8FEF+1215]
	RtlGetFullPathName_UEx [0x777F8FBD+1165]



<ipython-input-51-833086e0a04f>:23: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  name_elements = driver.find_elements_by_xpath("//div[@class ='ant-layout']" and "//div[@class = 'ant-row-flex RestaurantListRow___1SbZY']" and "//div[@class ='ant-col-24 RestaurantListCol___1FZ8V  ant-col-md-12 ant-col-lg-6']" and "//h6[@class = 'name___2epcT']")
<ipython-input-51-833086e0a04f>:24: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  url_elements = driver.find_elements_by_xpath("//a[contains(@href, '/ph/en/restaurant')]")


Restaurant                              Domino's Pizza - Espana Boulevard
URL                     https://food.grab.com/ph/en/restaurant/a-venet...
Latitude                                                                 
Longitude                                                                
Latitude & Longitude                                                     
Name: 0, dtype: object
OP 14.5995 120.9842
14.5995,120.9842
Restaurant              Pizza Telefono - UST [Available for LONG-DISTA...
URL                     https://food.grab.com/ph/en/restaurant/domino-...
Latitude                                                                 
Longitude                                                                
Latitude & Longitude                                                     
Name: 1, dtype: object
OP 14.5995 120.9842
14.5995,120.9842
Restaurant              Amici - Don Bosco [Available for LONG-DISTANCE...
URL                     https://food.grab.com/ph/en/restaurant/piz

Restaurant              Caruso Ristorante Italiano - LRI Design Plaza ...
URL                     https://food.grab.com/ph/en/restaurant/cali-pi...
Latitude                                                                 
Longitude                                                                
Latitude & Longitude                                                     
Name: 20, dtype: object
OP 14.5995 120.9842
14.5995,120.9842
Restaurant              Bossing Kambal Pizza and Silog Meals - R. Papa...
URL                     https://food.grab.com/ph/en/restaurant/caruso-...
Latitude                                                                 
Longitude                                                                
Latitude & Longitude                                                     
Name: 21, dtype: object
OP 14.5995 120.9842
14.5995,120.9842
Restaurant              The Perfect Pint - Greenbelt 2 [Available for ...
URL                     https://food.grab.com/ph/en/restaurant/b

...

Restaurant              Hakata Ton-Ichi - Ayala North Exchange [Availa...
URL                     https://food.grab.com/ph/en/restaurant/gulu-gu...
Latitude                                                                 
Longitude                                                                
Latitude & Longitude                                                     
Name: 60, dtype: object
OP 14.5995 120.9842
14.5995,120.9842
Restaurant              UCC Park Cafe - Magnolia [Available for LONG-D...
URL                     https://food.grab.com/ph/en/restaurant/hakata-...
Latitude                                                                 
Longitude                                                                
Latitude & Longitude                                                     
Name: 61, dtype: object
OP 14.5995 120.9842
14.5995,120.9842
Restaurant                                       Jim's Recipe - Masangkay
URL                     https://food.grab.com/ph/en/restaurant/u

Restaurant              Mitasu - Banawe [Available for LONG-DISTANCE D...
URL                     https://food.grab.com/ph/en/restaurant/tempura...
Latitude                                                                 
Longitude                                                                
Latitude & Longitude                                                     
Name: 80, dtype: object
OP 14.5995 120.9842
14.5995,120.9842
Restaurant              Nabe - Banawe [Available for LONG-DISTANCE DEL...
URL                     https://food.grab.com/ph/en/restaurant/mitasu-...
Latitude                                                                 
Longitude                                                                
Latitude & Longitude                                                     
Name: 81, dtype: object
OP 14.5995 120.9842
14.5995,120.9842
Restaurant              Sushi Bake by Maja - San Antonio Village [Avai...
URL                     https://food.grab.com/ph/en/restaurant/n

Restaurant                                           Rojiu Sunaku - Tondo
URL                     https://food.grab.com/ph/en/restaurant/sushibo...
Latitude                                                                 
Longitude                                                                
Latitude & Longitude                                                     
Name: 100, dtype: object
OP 14.5995 120.9842
14.5995,120.9842
Restaurant              Sarariman - Little Baguio [Available for LONG-...
URL                     https://food.grab.com/ph/en/restaurant/rojiu-s...
Latitude                                                                 
Longitude                                                                
Latitude & Longitude                                                     
Name: 101, dtype: object
OP 14.5995 120.9842
14.5995,120.9842
Restaurant              Matsuri Bento - South Triangle [Available for ...
URL                     https://food.grab.com/ph/en/restaurant

Restaurant              PaKaMuRa - Champaca Street [Available for LONG...
URL                     https://food.grab.com/ph/en/restaurant/takoich...
Latitude                                                                 
Longitude                                                                
Latitude & Longitude                                                     
Name: 119, dtype: object
OP 14.5995 120.9842
14.5995,120.9842
Restaurant              Sheikh's Kebab - Poblacion [Available for LONG...
URL                     https://food.grab.com/ph/en/restaurant/pakamur...
Latitude                                                                 
Longitude                                                                
Latitude & Longitude                                                     
Name: 120, dtype: object
OP 14.5995 120.9842
14.5995,120.9842
Restaurant              Kyoto Chaya Japanese Cafe - MET Live [Availabl...
URL                     https://food.grab.com/ph/en/restaurant

KeyboardInterrupt: 

In [54]:
maindb.to_csv("OP - Task 2 data.csv")